In [12]:
import numpy as np
import sympy as sp
from numba import njit

In [42]:
input = {}
input['vx'] = (np.random.random((1000,1000,1000))).reshape((-1,))
input['vy'] = (np.random.random((1000,1000,1000))).reshape((-1,))
input['vz'] = (np.random.random((1000,1000,1000))).reshape((-1,))



In [55]:
vxP, vxNP, vyP, vyNP, vzP, vzNP = sp.symbols(('vxP', 'vxNP', 'vyP', 'vyNP', 'vzP', 'vzNP'))
expr_ps = ((vxP-vxNP)*(vxP-vxNP) + (vyP-vyNP)*(vyP-vyNP) + (vzP-vzNP)*(vzP-vzNP))*(vxP-vxNP)
fx = njit(sp.lambdify((vxP, vxNP, vyP, vyNP, vzP, vzNP),expr_ps,'numpy'))

@njit
def f(i,ip,vx,vy,vz,fx=fx):
    vxP = vx[ip]
    vxNP = vx[i]
    vyP = vy[ip]
    vyNP = vy[i]
    vzP = vz[ip]
    vzNP = vz[i]
    return fx(vxP, vxNP, vyP, vyNP, vzP, vzNP)

print(fx)
print(f)
    

CPUDispatcher(<function _lambdifygenerated at 0x7f97b622d8b0>)
CPUDispatcher(<function f at 0x7f97b60d2430>)


In [63]:
class Term:
    def __init__(self):
        vxP, vxNP, vyP, vyNP, vzP, vzNP = sp.symbols(('vxP', 'vxNP', 'vyP', 'vyNP', 'vzP', 'vzNP'))
        self.expr = ((vxP-vxNP)*(vxP-vxNP) + (vyP-vyNP)*(vyP-vyNP) + (vzP-vzNP)*(vzP-vzNP))*(vxP-vxNP)


fy = njit(sp.lambdify((vxP, vxNP, vyP, vyNP, vzP, vzNP),Term().expr,'numpy'))

@njit
def f3(i,ip,vx,vy,vz,fx=fy):
    vxP = vx[ip]
    vxNP = vx[i]
    vyP = vy[ip]
    vyNP = vy[i]
    vzP = vz[ip]
    vzNP = vz[i]
    return fx(vxP, vxNP, vyP, vyNP, vzP, vzNP)

print(fx)
print(f)

CPUDispatcher(<function _lambdifygenerated at 0x7f97b3e77c10>)
CPUDispatcher(<function f at 0x7f97b60d2430>)


In [53]:
@njit
def f2(i,ip,vx,vy,vz):
    vxP = vx[ip]
    vxNP = vx[i]
    vyP = vy[ip]
    vyNP = vy[i]
    vzP = vz[ip]
    vzNP = vz[i]
    return ((vxP-vxNP)*(vxP-vxNP) + (vyP-vyNP)*(vyP-vyNP) + (vzP-vzNP)*(vzP-vzNP))*(vxP-vxNP) 

In [25]:
@njit
def test_f4(l,dist:int, f, *quantities):
    acc = 0
    for i in range(l - dist):
        acc +=  f(i,i+dist, *quantities) 
    offset = l-dist
    for i in range(dist):
        acc +=   f(i+offset, i, *quantities)
    return acc

In [56]:
print(test_f4(len(input['vx']),int(10), f, input['vx'],input['vy'],input['vz']))
%timeit test_f4(len(input['vx']),int(10), f, input['vx'],input['vy'],input['vz'])

6554.442663145018
2.26 s ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
print(test_f4(len(input['vx']),int(10), f2, input['vx'],input['vy'],input['vz']))
%timeit test_f4(len(input['vx']),int(10), f2, input['vx'],input['vy'],input['vz'])

6554.442663145018
2.29 s ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
print(test_f4(len(input['vx']),int(10), f3, input['vx'],input['vy'],input['vz']))
%timeit test_f4(len(input['vx']),int(10), f3, input['vx'],input['vy'],input['vz'])

6554.442663145018
2.25 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
